现使用iris数据集，建立变量Sepal.Length、Sepal.Width、Petal.Length对变量Petal.Width的Lasso回归模型，并手动编写Python对Lasso算法进行实现

In [1]:
import pandas as pd
import numpy as np

iris=pd.read_csv("http://image.cador.cn/data/iris.csv")
x = iris.iloc[:,[0,1,2]]
y = iris.iloc[:,3]
x = x.apply(lambda x:(x - np.mean(x))/np.sqrt(np.sum((x - np.mean(x))**2))).values
y = (y - np.mean(y)).values

首先对自变量数据矩阵x进行中心标准化处理，对目标变量y进行去中心化处理。其次声明Lasso算法需要用到的变量，并进行初始化处理

In [2]:
#活动变量下标集合
m = x.shape[1]
active = []
max_steps = m + 1

#初始化回归系数矩阵
beta = np.zeros((max_steps, m))
eps = 2.220446e-16

# 非活动变量与残差的相关系数
C = []
Sign = []

#非活动变量下标集合
im = range(m)
inactive = range(m)
k = 0

#计算y与x的相关性
Cvec = np.matmul(y.T,x)

#被忽略的变量下标集合
ignores = []

当达到最大迭代次数或活动变量集A中的元素个数达到最大自变量个数的时候退出循环

当发现候选变量中的最佳相关系数接近于0时，也可退出循环

In [3]:
while k < max_steps and len(active) < m:
    C = Cvec[inactive]
    Cmax = np.max(np.abs(C))
    if Cmax < eps*100:
        print("最大的相关系数为0，退出循环\n")
        break
    new = np.abs(C) >= Cmax - eps
    C = C[np.logical_not(new)]
    new = np.array(inactive)[new]
    for inew in new:
        if np.linalg.matrix_rank(x[:,active+[inew]]) == len(active):
            ignores.append(inew)
        else:
            active.append(inew)
            Sign.append(np.sign(Cvec[inew]))
    
    active_len = len(active)
    exclude = active + ignores
    inactive=[]
    t0 = [inactive.append(v) if i not in exclude else None for i,v in enumerate(im)]
    xa = x[:,active]*Sign
    oneA = [1]*active_len
    A = np.matmul(np.matmul(oneA,np.linalg.inv(np.matmul(xa.T,xa))),oneA)**(-0.5)
    w = np.matmul(A*np.linalg.inv(np.matmul(xa.T,xa)),oneA)
    if active_len >= m:
        gamhat = Cmax/A
    else:
        a = np.matmul(np.matmul(x[:,inactive].T,xa),w)
        gam = np.array([(Cmax - C)/(A - a), (Cmax + C)/(A + a)])
        gamhat = np.min([np.min(gam[gam > eps]),Cmax/A])

    b1 = beta[k, active]
    z1 = np.array(-b1/(w*Sign))
    zmin = np.min(z1[z1 > eps].tolist()+[gamhat])
    gamhat = zmin if zmin < gamhat else gamhat
    beta[k + 1, active] = beta[k, active] + gamhat*w*Sign
    Cvec = Cvec - gamhat*np.matmul(np.matmul(xa,w).T,x)
    k=k+1

得到回归系数的估计矩阵beta

In [4]:
beta

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  8.65652655],
       [ 0.        ,  0.27627203,  8.93279858],
       [-2.09501133,  1.18554279, 11.29305357]])